Another Llava model, but in huggingface format that can be directly loaded using pure Transformers API. Pro: More consistent for codebase. Con: Might be slightly less accurate.

In [ ]:
%%bash
# Install the model
huggingface-cli download llava-hf/llava-1.5-7b-hf \
        --local-dir ~/gai/models/llava-1.5-7b-hf \
        --local-dir-use-symlinks False


In [ ]:
%%bash
# This requires the latest transformer version so activate ITT2 environment 
pip install -e ".[ITT2]"

In [2]:
## Generating Text from Images
from gai.gen.itt.Transformers_ITT import Transformers_ITT
config = {
    "type": "itt",
    "engine": "Transformers_ITT",
    "model_path": "models/llava-1.5-7b-hf",
    "model_name": "llava-1.5-7b",
}
import base64
encoded_string = ""
with open("./buses.jpeg", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode('utf-8')

itt = Transformers_ITT(config)
response = itt.create(
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "Describe what you see in this image."},
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{encoded_string}",
          },
        },
      ],
    }
  ],
  temperature=10e-9,
  stream=False
  )
print(response.choices[0].message.content)

Loading checkpoint shards: 100%|██████████| 3/3 [00:41<00:00, 13.87s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/roylai/miniconda/envs/ITT-2/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1e-08` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


ER:  
Describe what you see in this image.
ASSISTANT: The image features a bus stop with a large sign displaying bus routes and numbers. The sign is located near a tree, providing shade and a natural backdrop. The bus stop is situated in a park-like setting, with a few other trees in the background. The sign is prominently displayed, making it easy for passengers to find their desired route.


In [ ]:
## Streaming Text from Images
from gai.gen.itt.Transformers_ITT import Transformers_ITT
config = {
    "type": "itt",
    "engine": "Transformers_ITT",
    "model_path": "models/llava-1.5-7b-hf",
    "model_name": "llava-1.5-7b",
}
import base64
encoded_string = ""
with open("./buses.jpeg", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode('utf-8')

itt = Transformers_ITT(config)
response = itt.create(
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "Describe what you see in this image."},
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{encoded_string}",
          },
        },
      ],
    }
  ],
  temperature=10e-9,
  stream=True
  )
for chunk in response:
    print(chunk.choices[0].delta.content,end="",flush=True)